# Import Modules

In [23]:
from Dataload import dataload

import pandas as pd

from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils
from constant import EMOTIPATH,EMOTIFACEPATH
from src import detect_faces, show_bboxes
from PIL import Image
import numpy as np
from torchvision import transforms
import os
import torch
from tqdm.notebook import tqdm


In [24]:
class Raw_Data():
    def __init__(self,csv_file,sub_csv_file=None,
                 base_path_v=None,frame_num=25,embedding=False):
        
        self.max_frame_num=25
        self._table = pd.read_csv(csv_file,delimiter=' ')
        if sub_csv_file is None:
            self._table_embedding=None
        else:
            self._table_embedding=pd.read_csv(sub_csv_file)
        self.frame_num = frame_num
        self._base_path_v=base_path_v
        self.embedding=embedding
        self.transform=transforms.Compose([
                    transforms.Resize((256,256)),
            
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
        self.endPad=self.transform(Image.new(mode='RGB', size=(256,256), color=0))    

       
    def __len__(self):
        return len(self._table)

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path_v,self._table.Vid_name[idx]) #base path
        first=True          
        frame_raw_list=os.listdir(folder_name)
        frame_len=len(frame_raw_list)

        frame_raw_list=sorted(frame_raw_list)
       # print(frame_raw_list)
        frame_list=[]
        
        if frame_len<self.frame_num:
            for index_0 in range(frame_len):
                frame_path=os.path.join(folder_name,frame_raw_list[index_0])               
                tempimg=Image.open(frame_path)       
                frame_list.append(self.transform(tempimg))
            
        else:    
            frame_index=(np.linspace(0,frame_len-1,self.frame_num,dtype=int))

            for index_2 in frame_index:
                frame_path=os.path.join(folder_name,frame_raw_list[index_2])
                tempimg=Image.open(frame_path)
                frame_list.append(self.transform(tempimg))
        while(len(frame_list)<self.frame_num):
            frame_list.append(self.endPad)
        frame_data=torch.stack(frame_list,dim=0)
            
        return (folder_name,frame_data,frame_raw_list)
    

def face_extraction(img,max_number):
        def toonePIL(x,max_number):
            new_im = Image.new('RGB', (64*max_number,64))

            x_offset = 0
            for im in x:
                new_im.paste(im, (x_offset,0))
                x_offset += im.size[0]
            return new_im

        bounding_boxes, landmarks = detect_faces(img)
        img_list=[]
        stacked_img=torch.zeros((5,3,64,64))
        transform=transforms.Resize((64,64))
        transform2=transforms.Compose([
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
        for box_index,(left,right,up,bottom,_) in enumerate(bounding_boxes):
            cropped_img=img.crop((left,right,up,bottom))
            cropped_img=transform(cropped_img)
            img_list.append(cropped_img)
            
            stacked_img[box_index]=transform2(cropped_img)
            if len(img_list)==max_number:
                break
        while len(img_list) !=max_number:
            END_PAD= Image.new(mode = "RGB", size = (64, 64), color =(0, 0, 0))
            img_list.append(END_PAD)
        img=toonePIL(img_list,max_number)
        pt=transform2(img)
        return img,pt,stacked_img
    
    


In [30]:
def save_frame(data,data_type="Test"):
    for i in tqdm(data):
        video_name=os.path.basename(i[0])
        file_name=video_name+'.pt'
        save_path=os.path.join("data/images_tensor",data_type)
        if not os.path.isdir(save_path):
                os.mkdir(save_path)

        torch.save(i[1], os.path.join(save_path,file_name))

In [26]:
def create_face(data,data_type="Test",face_num=5):
    transformation=transforms.Compose([
                 transforms.ToTensor(),   
                 transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
    for index in tqdm(data):
        video_name,_,frame_list=index
        frame_size=len(frame_list)
        save_stacked_pt_path=os.path.join("data/faces_tensor",data_type)
        save_image_path=os.path.join("data/faces",data_type,os.path.basename(video_name))
        if not os.path.isdir(save_image_path):
                 os.makedirs(save_image_path)
        if not os.path.isdir(save_stacked_pt_path):
                os.makedirs(save_stacked_pt_path)
        stacked_face=torch.empty(size=(frame_size,face_num,3,64,64))
        for frame_num,frame_name in enumerate(frame_list):

            path=os.path.join(video_name,frame_name)
            img=Image.open(path)
            try:
                face_cropped,pt,stacked_img=face_extraction(img,5)
            except:
                stacked_img=torch.zeros((5,3,64,64))
                face_cropped=Image.new(mode='RGB', size=(320,64), color=0)
                pt=transformation(face_cropped)
                print(video_name,frame_name)
                continue
            stacked_face[frame_num]=stacked_img
            face_cropped.save((os.path.join(save_image_path,frame_name)))
        torch.save(stacked_face,(os.path.join(save_stacked_pt_path,os.path.basename(video_name)+'.pt')))


In [27]:
Test_label=os.path.join("Test files.csv")
Test_frames=os.path.join("data/images","Test")
Test_Data=Raw_Data(Test_label,base_path_v=Test_frames,frame_num=25)


In [31]:
save_frame(Test_Data)

KeyError: 20

In [32]:
create_face(Test_Data)

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'data/images\\Test\\Test_6'

In [ ]:
save_frame